***Battle of Neighbourhoods - Week 1***

This notebook consist of two parts.

**1 Introduction/Business Problem**

   **1.a Discussion of the business problem and the audience who would be interested in this project.**

**2 Data Section**

**2.a What data is used?
   2.b Importing Libraries
   2.c Credentials and Core location
   2.d Search for hotel & restaurant within 1 KM
   2.e Location of Hotels**

**1. Introduction/Business Problem**

***Discussion of the business problem and the audience who would be interested in this project.***


***Something about the place - Hitech city,Hyderabad***


***I am from Hyderabad,the most busy place, largest city of the Indian state of Telangana***

***Opening of Hotel/Restaurant***

Coming down to business problem,I would like to open a hotel/restaurant near Hitech city.As it is a famous spot,there is already lots of attention towards it.I know there will be many competitors in terms of hotel and restaurant.But keeping them in mind,i need to locate my hotel in place where more people are attracted and comfortable for a stay and a good meal.I want to bring foreign and local peoples attention towards my new hotel.I would like to flavour my retaurant recipe with Italian,American,typical south & north indian foods to grab their taste.

The challenge is to find a suitable location for opening a new hotel / restaurant attracted to all local and foreign people in the centre of all famous venues.


Expected / Interested Audience: Most of the People near by who work in IT companies in surrounding areas can be expected to visit this hotel/Restaurant because as we see more people around who saves time,they rush to work. 90% of the people of them does that and rest of the people visit depending upon ratings, interest in  that particular taste of food.

***2.Data section***


***2.a What data is used?***

We will be completely working on Foursquare data to explore and try to locate our new hotel where more venues like church, temples, beach, museums, memorials that are present nearby.

How will we be solving using this data?

We will looking for midpoint area of venues to locate our new hotel.Before that our major focus will be on all venues present in and around the core place of hitech city.

Just a heads up on how many hotels are distributed now around Hitech City.We will perform some EDA on hotels & restaurants present in the particular spot.On furthur notebook we will use Foursquare data to determine other venues as well.

***2.b Importing Libraries***

In [14]:
# Import libraries
import numpy as np # data in a vectorized manner manipulation
import pandas as pd # data analsysis
import requests # HTTP library
!pip install beautifulsoup4
from bs4 import BeautifulSoup # scraping library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # JSON files manipulation

from sklearn.cluster import KMeans # clustering algorithm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries loaded")

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries loaded


***2.c Credentials and Core location***

In [15]:
CLIENT_ID = 'BQZSYSMJTC3M4RJYSXEIYFIWPM0ROCCZWDQCBRFPIWHTNXLA' # your Foursquare ID
CLIENT_SECRET = 'NGJSGD54MX4BE4O5CLHHOXBGRNQR5RDRDU3X4OC5NJJCITM1 ' # your Foursquare Secret
VERSION = '20191028'
LIMIT = 150

address = "Hitech City , India"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude =location.latitude   #17.4435  # location.Latitude 
longitude =location.longitude #78.3772  # location.longitude 

Place='Hitech City location : {},{}'.format(latitude,longitude)
print(Place)

Hitech City location : 17.4698143,78.3853779


***2.d Search for hotel & restaurant within 1 KM***

In [16]:
search_query = 'hotel'
search_query_res = 'restaurant'

radius = 1000
url_hotel = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url_restaurant = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_res, radius, LIMIT)
#url

Send the GET Request of hotel & restaurants and examine the results

In [17]:
results_hotel = requests.get(url_hotel).json()
results_restaurant = requests.get(url_restaurant).json()
#results_hotel

Get relevant part of JSON and transform it into a pandas dataframe


In [18]:
# assign relevant part of JSON to venues
venues_hotel = results_hotel['response']['venues']
venues_restaurant = results_restaurant['response']['venues']

# tranform venues into a dataframe and merging both data
dataframe_hotel = json_normalize(venues_hotel)
dataframe_restaurant = json_normalize(venues_restaurant)

dataframe = pd.concat([dataframe_hotel,dataframe_restaurant])

print("There are {} restaurants and hotels at Hitech City".format(dataframe.shape[0]))

C:\Users\srilatha\Anaconda3\Anaconda Navigator1\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
C:\Users\srilatha\Anaconda3\Anaconda Navigator1\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


There are 3 restaurants and hotels at Hitech City


Define information of interest and filter dataframe

In [20]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

  
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered
hotels_df=dataframe_filtered[['name','categories','distance','lat','lng','id']]
hotels_df.head()

,name,categories,distance,lat,lng,id
0,Hotel Siva Parvathi,Indian Restaurant,395,17.472008,78.388306,5222c44d498ea00643f64834
0,Messeleistro Restaurant,Restaurant,409,17.472739,78.383034,58e5f9a480e1af764824dda5
1,Fisher Men Restaurant,Restaurant,587,17.474392,78.388130,52fb989a498e8c5b1460a4b6


In [22]:
hotels_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the Hitech city 

# add a red circle marker to represent the core location of Hitech city
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hitech City',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6
).add_to(hotels_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(hotels_df.lat, hotels_df.lng, hotels_df.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(hotels_map)

# display map
hotels_map